In [20]:
from tqdm import tqdm
import re
import numpy as np
from collections import defaultdict
import heapq as heap

with open('data.txt') as file:
    data = file.read().splitlines()

In [21]:
valves = {}

for valve in data:
    v = {}
    a,b = valve.split(';')

    v['open'] = False

    name = a.split(' ')[1]
    v['rate'] = int(a.split(' ')[-1].split('=')[-1])
    
    nexts = b.split(', ')
    nexts[0] = nexts[0].split(' ')[-1]

    v['next'] = nexts
    v['name'] = name
    
    valves[name] = v

In [ ]:
a b c d c d c d

In [32]:
cache = {}

def get_best_path(key, open_valves, minutes_left):

    cache_key = (key, tuple(set(open_valves)), minutes_left)
    if cached := cache.get(cache_key, None):
        return cached

    alts = []

    if minutes_left == 0 or len(valves.items()) == len(open_valves):
        return 0

    if key not in open_valves:
        release = (minutes_left - 1) * valves[key]['rate'] + get_best_path(key, open_valves + [key], minutes_left=minutes_left-1)
        alts.append(release)

    for valve in valves[key]['next']:
        release = get_best_path(valve, open_valves, minutes_left=minutes_left-1)
        alts.append(release)

    total_score = max(alts)
    cache[cache_key] = total_score
    return total_score


zero_rates = [key for key, val in valves.items() if val['rate'] == 0]

get_best_path('AA', zero_rates, 30)

1651

In [33]:
cache = {}


def get_best_path_2(key, elephant, open_valves, minutes_left):

    cache_key = (tuple(set([key, elephant])), tuple(set(open_valves)), minutes_left)
    if cached := cache.get(cache_key, None):
        return cached

    alts = []

    if minutes_left == 0 or len(valves.items()) == len(open_valves):
        return 0

    if key not in open_valves:

        open_score = (minutes_left - 1) * valves[key]["rate"]

        # Elephant open also
        if elephant not in open_valves and elephant != key:

            release = (
                open_score
                + (minutes_left - 1) * valves[elephant]["rate"]
                + get_best_path_2(
                    key, elephant, open_valves + [key, elephant], minutes_left=minutes_left - 1
                )
            )

            alts.append(release)

        # Open and all elephant moves
        for elephant_move in valves[elephant]["next"]:
            release = open_score + get_best_path_2(
                key, elephant_move, open_valves + [key], minutes_left=minutes_left - 1
            )
            alts.append(release)


    for new_valve in valves[key]["next"]:

        # Elephant open

        if elephant not in open_valves:
            release = (
                (minutes_left - 1) * valves[elephant]["rate"]
                + get_best_path_2(
                    new_valve, elephant, open_valves + [elephant], minutes_left=minutes_left - 1
                )
            )

            alts.append(release)

        # Open and all elephant moves
        for elephant_move in valves[elephant]["next"]:
            release = get_best_path_2(
                new_valve, elephant_move, open_valves, minutes_left=minutes_left - 1
            )
            alts.append(release)



    total_score = max(alts)
    cache[cache_key] = total_score
    return total_score


zero_rates = [key for key, val in valves.items() if val["rate"] == 0]

get_best_path_2("AA", "AA", zero_rates, 26)


1707